In [1]:
import pandas as pd
import numpy as np
import datetime

Load Raw Data, Scores Data, and on Base Data for Each Year

In [2]:
pitchers_23 = pd.read_csv('data/pitches_23.csv')
pitchers_23['year'] = 2023
pitchers_22 = pd.read_csv('data/pitches_22.csv')
pitchers_22['year'] = 2022
pitchers_21 = pd.read_csv('data/pitches_21.csv')
pitchers_21['year'] = 2021

pitchers = pd.concat([pitchers_23, pitchers_22, pitchers_21])

In [3]:
score_21 = pd.read_csv('data/gameid_ab_score_21.csv')
score_21['year'] = 2021
score_22 = pd.read_csv('data/gameid_ab_score_22.csv')
score_22['year'] = 2022
score_23 = pd.read_csv('data/gameid_ab_score_23.csv')
score_23['year'] = 2023

scores = pd.concat([score_21, score_22, score_23])

In [4]:
on_base_21 = pd.read_csv('data/pitch_data_2021.csv')
on_base_21['year'] = 2021
on_base_22 = pd.read_csv('data/pitch_data_2022.csv')
on_base_22['year'] = 2022
on_base_23 = pd.read_csv('data/pitch_data_2023.csv')
on_base_23['year'] = 2023

on_base = pd.concat([on_base_21, on_base_22, on_base_23])

Subset Data to only include pitchers of interest

In [5]:
pitchers_to_keep = pd.read_csv('data/pitcher_list.csv') #pitcher subset list from David's Subset2.R
pitcher_subset = pitchers[pitchers['pitcher'].isin(pitchers_to_keep['pitcher'].values)]

Combine all Datasets on GameId, AB, and Year

In [6]:
merged_scores = pd.merge(pitcher_subset, scores, on = ['gameid', 'ab', 'year'], how='left')
merged_scores[['visscore', 'homscore']] = merged_scores[['visscore', 'homscore']].ffill() #front fill na values (assumes score hasn't changed)
final_dataset = pd.merge(merged_scores, on_base, on = ['gameid', 'ab', 'year', 'pitchnum'], how='left')
final_dataset = final_dataset.drop(['Unnamed: 0_x', 'GameDate_y', 'Unnamed: 0_y'], axis = 1).rename({'GameDate_x':'GameDate'}, axis = 1)

Create Pitch Type Column (Fastball or Breaking Ball)

In [7]:
pitch_mapping = {
    "Four-seam FB": "Fastball",
    "Cutter": "Fastball",
    "Sinker": "Fastball",
    "Fastball": "Fastball",
    "Two-seam FB": "Fastball",
    
    "Knuckle Curve": "Breaking Ball",
    "Slow Curve": "Breaking Ball",
    "Sweeper": "Breaking Ball",
    "Slider": "Breaking Ball",
    "Curveball": "Breaking Ball"
}
final_dataset['pitch_type'] = final_dataset['pitchname_desc'].map(pitch_mapping)

In [8]:
final_dataset.to_csv('pitcher_subset.csv', index=False)